In [5]:
import requests
import pandas as pd
import sqlite3
import os
from dotenv import load_dotenv

📊 Indicadores Macroeconómicos (EE. UU.)

Para enriquecer el modelo de credit scoring y capturar el contexto económico en el momento de otorgamiento de los préstamos, se incorporaron indicadores macroeconómicos oficiales de Estados Unidos.

Se utilizaron las siguientes variables:

- Inflación (CPI): Índice de Precios al Consumidor, mensual. Refleja la evolución del nivel general de precios. Permite al modelo ajustar el riesgo considerando la pérdida de poder adquisitivo de los prestatarios.

- Producto Interno Bruto (PIB): Trimestral (resampleado a mensual mediante forward fill). Indica la salud económica general del país. Un crecimiento económico sostenido suele asociarse con menor riesgo crediticio.

- Tasa de desempleo: Mensual. Mide la proporción de personas activas sin empleo. Altos niveles de desempleo aumentan la probabilidad de incumplimiento de los préstamos.

- Tasa de Fondos Federales (FEDFUNDS): Mensual. Representa el costo del dinero en la economía y afecta tasas de interés de préstamos. Ayuda a modelar cómo cambios en política monetaria pueden influir en la capacidad de pago de los clientes.

- Gasto en Consumo Personal (PCE): Mensual. Mide el gasto de los hogares en bienes y servicios, ajustado por inflación. Es un indicador de la demanda interna y la solvencia de los consumidores.

- Crédito al Consumidor Total (TOTALSL): Trimestral (resampleado a mensual mediante forward fill). Refleja el endeudamiento de los hogares. Un alto nivel de crédito total puede indicar mayor exposición al riesgo de impago.

Fuente de datos

Los datos fueron obtenidos a través de la API de FRED (Federal Reserve Economic Data), administrada por el Federal Reserve Bank of St. Louis, que provee series económicas oficiales y actualizadas del gobierno de Estados Unidos.

Proceso de obtención

- Se accedió a la API de FRED mediante una clave personal.

- Se descargaron las series históricas correspondientes al período de análisis del dataset de Lending Club.

- Se procesaron y transformaron en formato tabular, adaptando la frecuencia mensual cuando fue necesario (forward fill para variables trimestrales).

- Finalmente, los indicadores macroeconómicos se integraron al proyecto para su posterior unión con los préstamos según la fecha de emisión (issue date).

Justificación de inclusión

La incorporación de estas variables permite que el modelo considere el entorno económico general, capturando factores que afectan la capacidad de pago y el comportamiento financiero de los prestatarios. Esto mejora la capacidad explicativa y predictiva del análisis de riesgo crediticio.

In [6]:
api_key = os.getenv("FRED_API_KEY") # para obtener la informacion tenemos que añadir nuestra api_key 

confirmamos dedes que años vamos a tomar datos, de acuerdo al DF principal que es el que manejamos en el EDA, tenemos que el año incial es el 2010 y el final es el 2018

In [ ]:
API_KEY = os.getenv("FRED_API_KEY")
BASE_URL = "https://api.stlouisfed.org/fred/series/observations"

# Verificar que la API_KEY se cargó correctamente
print(f"API_KEY cargada: {API_KEY[:10]}..." if API_KEY else "ERROR: API_KEY no encontrada")

# Función para descargar series económicas desde la API de FRED (Banco de la Reserva Federal de St. Louis)
# y convertirlas en DataFrames de pandas con columnas 'date' y 'value'.

def get_fred_series(series_id, start_date="2010-01-01"):
    params = {
        "series_id": series_id,
        "api_key": API_KEY,
        "file_type": "json",
        "observation_start": start_date
    }
    response = requests.get(BASE_URL, params=params)
    
    # Debug: mostrar la respuesta si hay error
    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        return None
    
    data = response.json()
    
    # Verificar si contiene 'observations'
    if "observations" not in data:
        print(f"Respuesta inesperada para {series_id}:")
        print(data)
        return None
    
    data = data["observations"]
    
    df = pd.DataFrame(data)
    df["date"] = pd.to_datetime(df["date"])
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    
    return df[["date", "value"]]

Descargamos las series


inflation = get_fred_series("CPIAUCSL")
inflation.rename(columns={"value": "ME_inflation_cpi"}, inplace=True)

gdp = get_fred_series("GDP")
gdp.rename(columns={"value": "ME_gdp"}, inplace=True)

unemployment = get_fred_series("UNRATE")
unemployment.rename(columns={"value": "ME_unemployment_rate"}, inplace=True)

fedfunds = get_fred_series("FEDFUNDS")
fedfunds.rename(columns={"value": "ME_fed_funds_rate"}, inplace=True)

pce = get_fred_series("PCE")
pce.rename(columns={"value": "ME_pce"}, inplace=True)

total_credit = get_fred_series("TOTALSL")
total_credit.rename(columns={"value": "ME_total_consumer_credit"}, inplace=True)
# GDP y Total Credit resampleados a mensual
gdp_monthly = gdp.set_index("date").resample("MS").ffill().reset_index()
total_credit_monthly = total_credit.set_index("date").resample("MS").ffill().reset_index()

# Merge usando las versiones mensuales
macro_df = inflation.merge(gdp_monthly, on="date", how="left")
macro_df = macro_df.merge(unemployment, on="date", how="left")
macro_df = macro_df.merge(fedfunds, on="date", how="left")
macro_df = macro_df.merge(pce, on="date", how="left")
macro_df = macro_df.merge(total_credit_monthly, on="date", how="left")

# Finalmente alinear fechas y forward-fill por si queda algún NaN
macro_df = macro_df.set_index("date").resample("MS").ffill().reset_index()

print(macro_df.head(12))


ERROR: API_KEY no encontrada


## Ajuste y Sincronización de Variables Macroeconómicas
Para enriquecer el conjunto de datos de Ladin Club con indicadores de la Reserva Federal (FRED), se implementó un proceso de alineación temporal y tratamiento de frecuencias para garantizar la validez estadística del análisis:
1. Homogeneización de Frecuencias: Dado que el dataset contiene variables con distintas periodicidades (mensuales como el Desempleo e Inflación, y trimestrales como el PIB), se utilizó una técnica de resampling mensual con Forward Fill (ffill). Esto permitió expandir los datos trimestrales para que cada observación mensual del crédito cuente con un marco macroeconómico de referencia.
2. Mitigación del Sesgo de Anticipación (Look-ahead Bias): En los modelos de riesgo crediticio, utilizar datos económicos del mismo mes de la observación genera resultados poco realistas, ya que los indicadores macroeconómicos se publican con retraso. Para corregir esto, se aplicó una técnica de rezago temporal (Time Lagging):
- Variables Mensuales: Se aplicó un rezago de -1 mes (t-1), asumiendo que indicadores como la tasa de desempleo o la inflación son conocidos por el mercado poco después de cerrar el mes.
- Variables Trimestrales: Se aplicó un rezago de -3 meses (t-3). Esto asegura que el impacto del ciclo económico en el comportamiento de pago de los clientes se analice utilizando información que ya era pública y oficial en el momento del evento crediticio.
3. Resultado: Esta estructura transforma el análisis de uno puramente descriptivo a uno predictivo, permitiendo evaluar cómo las condiciones económicas previas influyen en la probabilidad de incumplimiento (default) actual de los préstamos.

In [9]:
def prepare_monthly(df):
    return df.set_index("date").resample("MS").ffill()

inf_m = prepare_monthly(inflation)
unemp_m = prepare_monthly(unemployment)
fed_m = prepare_monthly(fedfunds)
pce_m = prepare_monthly(pce)
gdp_m = prepare_monthly(gdp)
credit_m = prepare_monthly(total_credit)

# 2. Unir todas en un DataFrame temporal
macro_df = pd.concat([inf_m, gdp_m, unemp_m, fed_m, pce_m, credit_m], axis=1)

# 3. Aplicar rezagos diferenciados
# Variables Mensuales: Rezago de 1 mes (datos que se publican rápido)
cols_mensuales = ["ME_inflation_cpi", "ME_unemployment_rate", "ME_fed_funds_rate", "ME_pce"]
macro_df[cols_mensuales] = macro_df[cols_mensuales].shift(1)

# Variables Trimestrales: Rezago de 3 meses (PIB y Crédito Total)
# Esto asegura que en cualquier mes, solo veas el dato del trimestre que ya cerró
cols_trimestrales = ["ME_gdp", "ME_total_consumer_credit"]
macro_df[cols_trimestrales] = macro_df[cols_trimestrales].shift(3)

# 4. Limpiar y resetear índice para el merge final
macro_df = macro_df.reset_index()

# Verificación visual: Los primeros meses tendrán NaNs por el desplazamiento
print("Primeras filas con rezagos aplicados:")
print(macro_df.head(5))

AttributeError: 'NoneType' object has no attribute 'set_index'

In [10]:
conn = sqlite3.connect(r"C:\Users\User\Documents\GITHUB\final_project_creditscoring\Data\credit_scoring.db")
cursor = conn.cursor()

# Guardar el DataFrame como tabla 'macro_data', reemplazando si ya existe
macro_df.to_sql("macro_data", conn, if_exists="replace", index=False)

print("Macro data guardada correctamente en credit_scoring.db -> tabla 'macro_data'")

OperationalError: unable to open database file

In [11]:


# Leer la tabla main_table
main_table = pd.read_sql_query("SELECT * FROM main_table", conn)

# Convertir issue_d a datetime
main_table['issue_d'] = pd.to_datetime(main_table['issue_d'])


# Hacemos merge de main_table con macro_df
merged_df = main_table.merge(
    macro_df, 
    left_on='issue_d', 
    right_on='date', 
    how='left'
)

# eliminamos la columna duplicada 'date' de macro_df
merged_df.drop(columns=['date'], inplace=True)

# Guardar el DataFrame resultante en la base de datos
merged_df.to_sql("main_table", conn, if_exists="replace", index=False)

# Cerrar conexión
conn.close()

NameError: name 'conn' is not defined